In [3]:
from dotenv import load_dotenv
import os
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI
import json
import pandas as pd

load_dotenv()

HF_API_TOKEN = os.environ["HF_API_TOKEN"]
GOOGLE_API_KEY = os.environ["GOOGLE_API_KEY"]

In [4]:
llm = ChatGoogleGenerativeAI(
    model="gemma-3-27b-it",
    temperature=0,
    google_api_key=GOOGLE_API_KEY
)

output_parser = StrOutputParser()

In [34]:
# KTAS 성인 고려사항 정리
with open ('../data/KTAS_성인.json', 'r', encoding='utf-8-sig') as f:
    KTAS_adult = json.load(f)

In [35]:
# KTAS 성인 고려사항 별 설명
KTAS_adult_description = pd.read_csv('../data/KTAS_고려사항_설명.csv', encoding='utf-8-sig')
KTAS_adult_description.fillna('', inplace=True)
KTAS_adult_description

,consideration,description
0,무의식,의식수준 Coma 또는 AVPU가 U(Unresponsive. 무반응) 또는 GCS...
1,의식변화,"의식수준 Drowsy/Lethargy, Stupor, Semi-coma 또는 AVP..."
2,쇼크,"약한 맥박, 심각한 빈맥 또는 서맥, 의식수준의 저하, 패혈성 쇼크처럼 홍조를 띠고..."
3,혈역학적 장애,"설명되지 않는 빈맥, 기립성 저혈압, 저혈압 의심"
4,비정상 맥박수지만 혈역학적으로 안정,정상 활력징후의 상한치 또는 하한치를 보이는 경우
5,중증 호흡곤란,산소포화도 <90% (급성)
6,중등도 호흡곤란,산소포화도 <92% (급성) 혹은 <90% (만성)
7,경증 호흡곤란,산소포화도 92-94%
8,"열, 면역저하 상태","중성구감소증 (또는 중성구감소증 의증), 항암 화학 요법 또는 스테로이드를 포함한 ..."
9,패혈증 의증,3개 이상의 SIRS 기준을 만족


In [45]:
for item in KTAS_adult:
    for consideration in item['primary']:
        description = KTAS_adult_description[KTAS_adult_description['consideration'] == consideration['consideration']].values[0][1]
        
        consideration['description'] = description


In [52]:
for item in KTAS_adult:
    for consideration in item['secondary']:
        if consideration['consideration'] == '중증 탈수':
            consideration['description'] = '고전적인 탈수 징후를 동반하는 심각한 체액 소실 및 쇼크의 증상과 징후'
        elif consideration['consideration'] == '중등도 탈수':
            consideration['description'] = '건조한 점막, 빈맥, 피부 탄력도 저하 및 소변량 감소'
        elif consideration['consideration'] == '경증 탈수':
            consideration['description'] = '안정된 활력징후를 보이나 목마름이 심해지고 소변색이 진해짐. 수분 섭취 감소나 체액 소실의 병력이 동반됨'
        elif consideration['consideration'] == '탈수 가능성 있음':
            consideration['description'] = '탈수 증상은 없으나 체액 소실이 지속되거나 경구 수분 섭취에 어려움이 동반되는 경우'

In [55]:
with open('../data/KTAS_성인_with_description.json', 'w', encoding='utf-8-sig') as f:
    json.dump(KTAS_adult, f, ensure_ascii=False, indent=4)


In [5]:
with open('../data/KTAS_성인_with_description.json', 'r', encoding='utf-8-sig') as f:
    KTAS_adult_with_description = json.load(f)

In [6]:
# 고려사항 추가 설명
SIRS_standard = """- 열 (체온 >38°C 또는 <36°C)
- 심박수 >90회/분
- 호흡수 >20회/분
- WBC >12000 cells/mm3 또는 <4000 cells/mm3"""

pain_standard = """- 중심성 통증: 체강이나 내부 장기에서 기원하는 통증으로 생명이나 사지 소실의 위험과 관련이 있을 수 있음.
- 말초성 통증: 피부, 연조직, 근골격계에서 기원하는 통증 또는 심각한 질환일 가능성이 비교적 낮은 표재성 기관에서 기원하는 통증
- 급성 통증: 한 달 미만의 새롭게 발병한 통증
- 만성 통증: 이전과 같은 패턴이 반복되거나 지속되는 환자가 잘 알고있는 오래된 통증"""

hemorrhagic_disorders = "혈우병, 혈소판 감소증 등 지혈 기능에 이상이 있는 질병 또는 혈소판 억제제, 와파린 등 지혈을 억제하는 약물 복용"

In [7]:
# Triage note 샘플
primary_samples = pd.read_csv('../data/KTAS_adult_samples_primary.csv', encoding='utf-8-sig', index_col=0)
secondary_samples = pd.read_csv('../data/KTAS_adult_samples_secondary.csv', encoding='utf-8-sig', index_col=0)


In [8]:
primary_samples = primary_samples.T
secondary_samples = secondary_samples.T
secondary_samples


,Triage Note,주증상
Case 1,신장: -cm\n체중: -kg\n\n주호소:\n\nDisorientated (기간:...,착란
Case 2,"신장: -cm\n체중: -kg\n\n주호소:\nHeadache, Weakness i...",사지 약화 / 뇌졸중 증상
Case 3,신장: 181 cm\n체중: - kg\n\n주호소:\n\nAVF malfunctio...,의료 장비 문제
Case 4,주호소:\n\nSeizure like movement (zittering) (기간:...,발작
Case 5,신장: 158 cm \n체중: 61.7 kg \n\n주호소: \n\nVagin...,질 출혈
Case 6,"신장: – cm\n체중: – kg\n\n주호소:\n\nFacial palsy, le...",감각상실 / 이상감각
Case 7,"주호소:\n\nNausea and vomiting (기간: 3일, 발현 시기: -)...",구토 / 구역
Case 8,신장: 154cm\n체중: 71.4kg\n\n주호소:\n\nPresyncope (기...,실신 / 전실신
Case 9,"주호소:\n\nPalpitation (기간: 02일 23시간 06분, 발현 시기: ...",혈변 / 흑색변
Case 10,신장: – cm\n체중: – kg\n\n주호소:\n\nDrug intoxicatio...,물질 오용 / 중독


In [9]:
# 증상에 따른 고려사항 추출
def get_consideration(symptom, KTAS):
    for item in KTAS:
        if item['symptom'] == symptom:
            primary_considerations = item['primary'] # 1차 고려사항
            secondary_considerations = item['secondary'] # 2차 고려사항
            break
    
    return primary_considerations, secondary_considerations

### 테스트1: 특별한 지시 제공 X

In [12]:
# KTAS 분류 파이프라인

template_classification = """You are a specialized emergency department triage classifer. You will receive a patient’s <Triage Note> and you must choose exactly one item from a predefined list of <Considerations>.

Important Constraints:
- You must choose only one item from the list.
- Refer to the description of the consideration to make the best decision.
- Your answer must exactly match the corresponding item in the <Considerations> (including spacing, punctuation, letter case, etc.).
- Output only the 'consideration' and 'KTAS Level' without adding any additional text, explanations, or formatting.
- Output format: '[KTAS Level] consideration'

Read the following information carefully and determine which item from the <Considerations> best matches the information given.

<Triage Note>:
{triage_note} 

<Considerations>
{considerations}

{additional_info}"""

prompt_classification = PromptTemplate.from_template(template_classification)
chain_classification = prompt_classification | llm | output_parser

1차 고려사항

In [ ]:
for i, case in primary_samples.iterrows():
    
    triage_note = case['Triage Note']
    symptom = case['주증상']
    primary_considerations, secondary_considerations = get_consideration(symptom, KTAS_adult_with_description)

    # SIRS 기준, 통증 기준, 출혈성 질환 기준 제시
    additional_info = ''

    if any(item['consideration'] == s for item in primary_considerations for s in ['패혈증 의증', '전신염증반응증후군']):
        additional_info += f'<SIRS 기준>\n{SIRS_standard}\n\n'
    
    if any(item['consideration'] == s for item in primary_considerations for s in [
            '급성 중심성 중증 통증',
            '급성 중심성 중등도 통증',
            '급성 중심성 경증 통증',
            '만성 중심성 중증 통증',
            '만성 중심성 중등도 통증',
            '만성 중심성 경증 통증', 
            '급성 말초성 중증 통증',
            '급성 말초성 중등도 통증',
            '급성 말초성 경증 통증',
            '만성 말초성 중증 통증',
            '만성 말초성 경증/중등도 통증'
        ]):
        additional_info += f'<통증 기준>\n{pain_standard}\n\n'
    
    if any(item['consideration'] == s for item in primary_considerations for s in ['출혈성 질환 (생명 혹은 사지를 소실할 정도의 위급한 출혈)', '출혈성 질환 (중등도나 경도의 출혈)']):
        additional_info += f'<출혈성 질환 기준>\n{hemorrhagic_disorders}\n\n'

    # 1차 고려사항 분류
    primary_result = chain_classification.invoke({
        'triage_note': triage_note,
        'considerations': primary_considerations,
        'additional_info': additional_info
    })

    print(i)    
    print('1차: ', primary_result)
    print()
    

Case 8
1차:  [3] 출혈성 질환 (중등도나 경도의 출혈)

Case 9
1차:  [3] 출혈성 질환 (중등도나 경도의 출혈)

Case 10
1차:  [3] 급성 중심성 중등도 통증



In [64]:
for i, case in secondary_samples.iterrows():

    triage_note = case['Triage Note']
    symptom = case['주증상']
    primary_considerations, secondary_considerations = get_consideration(symptom, KTAS_adult_with_description)

    # 1차 고려사항 분류
    secondary_result = chain_classification.invoke({
        'triage_note': triage_note,
        'considerations': secondary_considerations,
        'additional_info': ''
    })

    print(i)
    print('2차: ', secondary_result)
    print()
    

Case 1
2차:  [2] 두통 또는 의식 상태의 변화를 동반한 급성 착란

Case 2
2차:  [3] 발병시간 24시간 초과

Case 3
2차:  [4] 의료 장비 문제, 증상이나 불편함이 없음

Case 4
2차:  [2] 발작 후 상태

Case 5
2차:  [3] 질 출혈 - 정상 활력징후

Case 6
2차:  [3] 새로 발병한 감각소실 / 이상감각

Case 7
2차:  [3] 경증 탈수

Case 8
2차:  [3] 전구증상 동반 또는 급격한 자세 변화시 발병

Case 9
2차:  [4] 적은 양의 직장 출혈

Case 10
2차:  [4] 알려진 저독성 물질



### 테스트 2: CoT

In [29]:
# KTAS 분류 파이프라인

template_classification = """You are a specialized emergency department triage classifer. You will receive a patient’s <Triage Note> and you must choose exactly one item from a predefined list of <Considerations>.

Important Constraints:
- You must choose only one item from the list.
- Refer to the description of the consideration to make the best decision.
- Think step by step following the instruction below:
Step 1: Analyze key findings from the <Triage Note>.
Step 2: Evaluate each item in the <Consideration> list against the <Traige Note>. Identify which conditions are met.
Step 3: Select the single most appropriate item. Reconfirm that it best reflects the patient's condition.

- Output your reasoning step, but end with the final answer in the format of '<FINAL ANSWER: [KTAS Level (number)] consideration (text)>'

Read the following information carefully and determine which item from the <Considerations> best matches the information given.

<Triage Note>:
{triage_note} 

<Considerations>
{considerations}

{additional_info}

Let's think step by step..."""

prompt_classification = PromptTemplate.from_template(template_classification)
chain_classification = prompt_classification | llm | output_parser

1차 고려사항

In [30]:
for i, case in primary_samples.iterrows():
    
    triage_note = case['Triage Note']
    symptom = case['주증상']
    primary_considerations, secondary_considerations = get_consideration(symptom, KTAS_adult_with_description)

    # SIRS 기준, 통증 기준, 출혈성 질환 기준 제시
    additional_info = ''

    if any(item['consideration'] == s for item in primary_considerations for s in ['패혈증 의증', '전신염증반응증후군']):
        additional_info += f'<SIRS 기준>\n{SIRS_standard}\n\n'
    
    if any(item['consideration'] == s for item in primary_considerations for s in [
            '급성 중심성 중증 통증',
            '급성 중심성 중등도 통증',
            '급성 중심성 경증 통증',
            '만성 중심성 중증 통증',
            '만성 중심성 중등도 통증',
            '만성 중심성 경증 통증', 
            '급성 말초성 중증 통증',
            '급성 말초성 중등도 통증',
            '급성 말초성 경증 통증',
            '만성 말초성 중증 통증',
            '만성 말초성 경증/중등도 통증'
        ]):
        additional_info += f'<통증 기준>\n{pain_standard}\n\n'
    
    if any(item['consideration'] == s for item in primary_considerations for s in ['출혈성 질환 (생명 혹은 사지를 소실할 정도의 위급한 출혈)', '출혈성 질환 (중등도나 경도의 출혈)']):
        additional_info += f'<출혈성 질환 기준>\n{hemorrhagic_disorders}\n\n'

    # 1차 고려사항 분류
    primary_result = chain_classification.invoke({
        'triage_note': triage_note,
        'considerations': primary_considerations,
        'additional_info': additional_info
    })

    print(i)    
    print('1차: ', primary_result)
    print()
    

Case 1
1차:  Step 1: Analyze key findings from the <Triage Note>.
The patient presents with a mental change for 3 days, is drowsy, and has elevated blood pressure (162/106 mmHg) and heart rate (107 bpm). They have a history of metastatic adenocarcinoma and cholangio CA. Their temperature is slightly elevated at 37.5°C, and oxygen saturation is 97%.

Step 2: Evaluate each item in the <Consideration> list against the <Triage Note>.
- **중증 호흡곤란 (KTAS 1):** Oxygen saturation is 97%, so this doesn't apply.
- **쇼크 (KTAS 1):** No signs of shock are present (normal pulse strength not mentioned, no severe tachycardia/bradycardia, no significant alteration of consciousness beyond drowsiness).
- **무의식 (KTAS 1):** The patient is drowsy, not unconscious.
- **중등도 호흡곤란 (KTAS 2):** Oxygen saturation is 97%, so this doesn't apply.
- **혈역학적 장애 (KTAS 2):** Blood pressure is elevated, and heart rate is slightly elevated, but not necessarily indicative of hemodynamic instability without further information.

2차 고려사항

In [31]:
for i, case in secondary_samples.iterrows():

    triage_note = case['Triage Note']
    symptom = case['주증상']
    primary_considerations, secondary_considerations = get_consideration(symptom, KTAS_adult_with_description)

    # 1차 고려사항 분류
    secondary_result = chain_classification.invoke({
        'triage_note': triage_note,
        'considerations': secondary_considerations,
        'additional_info': ''
    })

    print(i)
    print('2차: ', secondary_result)
    print()
    

Case 1
2차:  Step 1: Analyze key findings from the <Triage Note>.
The patient presents with disorientation for 3 hours and 29 minutes. The level of consciousness is alert, and vital signs are relatively stable (BP 114/78, HR 87, RR 16, Temp 36.4, SpO2 98%). There's no mention of headache. A flapping tremor was observed, and the patient was advised to undergo further examination at a secondary hospital. There is no significant past medical history, allergies, or medication use.

Step 2: Evaluate each item in the <Consideration> list against the <Triage Note>.
- '혈당 <54 mg/dl 이고 관련증상을 동반': The blood glucose level is not provided, so this cannot be assessed.
- '두통 또는 의식 상태의 변화를 동반한 급성 착란': The patient has acute disorientation (a change in mental status), but there is no mention of headache.
- '두통 또는 의식 상태의 변화를 동반하지 않는 급성 착란': The patient has acute disorientation (a change in mental status), and there is no mention of headache. This seems to fit the case.
- '만성, 평상시에 비해 변화 없음': The disorien